In [1]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

In [34]:
movies = pd.read_csv('ml-1m\movies.dat', sep = '::', header=None)
ratings = pd.read_csv(r'ml-1m\ratings.dat', sep = '::', header=None)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [41]:
movies.columns = ['movieId', 'title', 'genres']
ratings.columns = ['userId', 'movieId', 'rating', 'timestamp']

In [43]:
ratings.drop(['timestamp'], axis=1, inplace=True)

In [46]:
movies.head()

movieId                               title                        genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Drama
4        5  Father of the Bride Part II (1995)                        Comedy

In [45]:
ratings.head()

userId  movieId  rating
0       1     1193       5
1       1      661       3
2       1      914       3
3       1     3408       4
4       1     2355       5

In [47]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [48]:
movies_with_ratings

movieId                  title                       genres  userId  \
0              1       Toy Story (1995)  Animation|Children's|Comedy     1.0   
1              1       Toy Story (1995)  Animation|Children's|Comedy     6.0   
2              1       Toy Story (1995)  Animation|Children's|Comedy     8.0   
3              1       Toy Story (1995)  Animation|Children's|Comedy     9.0   
4              1       Toy Story (1995)  Animation|Children's|Comedy    10.0   
...          ...                    ...                          ...     ...   
1000381     3952  Contender, The (2000)               Drama|Thriller  5812.0   
1000382     3952  Contender, The (2000)               Drama|Thriller  5831.0   
1000383     3952  Contender, The (2000)               Drama|Thriller  5837.0   
1000384     3952  Contender, The (2000)               Drama|Thriller  5927.0   
1000385     3952  Contender, The (2000)               Drama|Thriller  5998.0   

         rating  
0           5.0  
1           4.0  
2           4.0  
3           5.0  
4           5.0  
...         ...  
1000381     4.0  
1000382     3.0  
1000383     4.0  
1000384     1.0  
1000385     4.0  

[1000209 rows x 5 columns]

In [49]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [50]:
ratings.rating.min()

1

In [51]:
ratings.rating.max()

5

In [52]:
reader = Reader(rating_scale=(1, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [54]:
trainset, testset = train_test_split(data, test_size=0.3)

In [58]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [59]:
test_pred = algo.test(testset)

In [60]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8654


0.8653805825049555